In [1]:
gROOT->ProcessLine(".L /home/rj/app/lib/libRadioScatter.so");
gROOT->ProcessLine("#include </home/rj/app/include/RadioScatter/RadioScatter.hh>");
gROOT->ProcessLine("#include </home/rj/app/include/RadioScatter/RSEventSummary.hh>");
gROOT->ProcessLine("#include </home/rj/app/include/RadioScatter/RadioScatterEvent.hh>");
gROOT->ProcessLine("#include </home/rj/app/include/RadioScatter/TUtilRadioScatter.hh>");
TString infile = "/home/rj/RadioScatter/doc/smallmultiscat_0MHz_10W_10ns.root";
auto ff=TFile::Open(infile, "READ");
TFile *outfile = new TFile("/home/rj/RadioScatter/outputfiles/voltage_time_angle.root", "RECREATE");
TTree *outtree = new TTree("tree", "test label");

int rxindex = 0;
int txindex = 0;
int antenna = 0;
int entry = 0;
auto tree=(TTree*)ff->Get("tree");

In [2]:
auto event= new RadioScatterEvent();
tree->SetBranchAddress("event", &event);
tree->GetEntry(29);
// tree->GetEntries();
cout << event->nrx;
c1 = TUtilRadioScatter::canvas();

3

In [ ]:
double bob=event->peakPowerW(0,0);
cout << "peakPowerW: " << bob << endl;

double joe=event->peakFreq(0,0);
cout << "peakFreq: " << joe << endl;

In [ ]:
int *triggers = new int[tree->GetEntries()];
for(int i = 0; i < tree->GetEntries(); i++){
    triggers[i] = 
}

In [ ]:
cout << tree->GetEntries() << endl;
cout << event->nrx << endl;
cout << event->ntx << endl;
    
cout << (tree->GetEntries() * event->nrx * event->ntx) << endl;
    

In [15]:
// Shuffled loops way (good for macros)
int totnumsigs = tree->GetEntries() * event->nrx * event->ntx;
auto *freqs = new double[totnumsigs];
auto *powers = new double[totnumsigs];
double frq;
double pow;

double thresh = 20e-6; 

cout << "Trigger threshold is: " << thresh << endl;
for(int k = 0; k < tree->GetEntries(); k++){
    tree->GetEntry(k); 
    int trig=0,num=0;
    for(int i=0;i<event->ntx;i++){
        for(int j=0;j<event->nrx;j++){
            trig=event->peakV(i,j)>=thresh?1:0;
            if(trig==1){
                num++;
                // Meat goes here
                frq = event->peakFreq(0,j);
                pow = event->peakPowerW(0,j);
                freqs[j]=frq;
                cout << "\n" << frq << endl;
                
                cout << "\nEvent " << k << " triggered on RX " << j << " at " << event->peakV(0,j) << "V ";
                cout << "with peak frequency " << freqs[j] << "MHz and power "<< pow << "W" << endl;
            }
            else{
                freqs[j] = 0;
            }
            if(num>=event->nrx){
                
            }
        }
    }
}

cout << "\n" << totnumsigs << endl;
for(int s = 0; s < totnumsigs; s++){
    cout << "freq " << s << ": " << freqs[s] << endl;
}

Trigger threshold is: 2e-05

0.0975

Event 0 triggered on RX 0 at 6.31918e-05V with peak frequency 0.0975MHz and power 7.98641e-11W

0.0975

Event 0 triggered on RX 1 at 6.46405e-05V with peak frequency 0.0975MHz and power 8.35678e-11W

0.0929672

Event 0 triggered on RX 2 at 6.13303e-05V with peak frequency 0.0929672MHz and power 7.52281e-11W

0.0979924

Event 1 triggered on RX 0 at 3.33513e-05V with peak frequency 0.0979924MHz and power 2.22461e-11W

0.0975

Event 1 triggered on RX 1 at 3.30791e-05V with peak frequency 0.0975MHz and power 2.18846e-11W

0.0925

Event 1 triggered on RX 2 at 3.19708e-05V with peak frequency 0.0925MHz and power 2.04426e-11W

0.0975

Event 2 triggered on RX 0 at 0.000333871V with peak frequency 0.0975MHz and power 2.22939e-09W

0.0975

Event 2 triggered on RX 1 at 0.000309828V with peak frequency 0.0975MHz and power 1.91987e-09W

0.0975

Event 2 triggered on RX 2 at 0.000280283V with peak frequency 0.0975MHz and power 1.57118e-09W

0.0925

Event 3 trigger

In [ ]:
// Function way (not macro compatible)

int triggered(double thresh, int n_antennas){
  int trig=0,num=0;
  for(int i=0;i<event->ntx;i++){
    for(int j=0;j<event->nrx;j++){
      trig=event->peakV(i,j)>=thresh?1:0;
      if(trig==1)num++;
      if(num>=event->nrx)return 1;
    }
  }
  return trig;
}

double thresh = 20e-6; 
// ~RoUgHlY~ a little more than background thermal noise RMS according to Steve is 5-10microVolts,
// thresh is set to 20microVolts
cout << "Trigger threshold is: " << thresh << endl;
for(int k = 0; k < tree->GetEntries(); k++){
    tree->GetEntry(k);
    if (triggered(thresh, 2) == 1){
        cout << "\nEvent " << k << " triggered: " << event->peakV(0,0) << "V";
    }
}
